In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from pycocotools.coco import COCO
import numpy as np
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)
from scipy.misc import imread
import matplotlib.patches as patches
from torch.utils.data import DataLoader
import cv2
import torch
from torch.autograd import  Variable
from faster_rcnn.utils.images import imshow
from faster_rcnn.utils.cython_bbox import bbox_overlaps

from faster_rcnn.fastrcnn.bbox_transform import bbox_transform, bbox_transform_inv
import cPickle
from faster_rcnn.rpn_msr.proposal_target_layer_2 import  ProposalTargetLayer
from torch.optim import SGD, RMSprop, Adam
from torch.optim.lr_scheduler import StepLR
from faster_rcnn.roi_pooling.modules.roi_pool import RoIPool
import gc
torch.backends.cudnn.enabled = False

In [3]:
with open('sorted_index', 'rb') as fp:
    sorted_index = cPickle.load(fp)

### Đọc dữ liệu từ MS COCO dataset


In [4]:
import os
import torchvision.transforms as transforms
from faster_rcnn.utils.datasets.mscoco.dataset import CocoData
from faster_rcnn.utils.datasets.data_generator import CocoGenerator
from faster_rcnn.utils.datasets.data_generator import Enqueuer

dataDir = './data/mscoco'
dataType = 'train2014'
annFile='%s/annotations/instances_%s.json'%(dataDir,dataType)
pre_proposal_folder = './data/mscoco/coco_proposals/MCG/'
batch_size = 3

images_dir = os.path.join(dataDir,'images', dataType)
cap = CocoData(root = images_dir, annFile = annFile)

data_gen = CocoGenerator(data=cap, sorted_index=sorted_index, batch_size=batch_size)
queue = Enqueuer(generator=data_gen)
queue.start(max_queue_size=10, workers=2)
t = queue.get()


loading annotations into memory...
Done (t=9.76s)
creating index...
index created!


Thử hiển thị ảnh cùng các bounding boxes

In [5]:
from faster_rcnn.faster_rcnn_2 import RPN, FastRCNN


### Tính toán feed-forward


Chúng ta sử dụng một ảnh có kích thước đầu vào là  `(width , height) = (600, 800)`

Input:
    - im_data : 
        kích thước : (batch_size, dim, witdh, height)
    - ground_boxes: 
        kích thước (n, 4)
        

In [6]:
categories = ['__background__'] + [x['name'] for x in cap.coco.loadCats(cap.coco.getCatIds())]

In [7]:
proposal_target_layer = ProposalTargetLayer(len(categories))


In [8]:
net = FastRCNN(categories, debug=False)
net.cuda()
net.train()

INFO:faster_rcnn.utils.datasets.mscoco.dataset:need more than 0 values to unpack
/opt/conda/envs/pytorch/lib/python2.7/site-packages/torch/nn/parallel/data_parallel.py:24: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
faster_rcnn/network.py:26: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.conv.weight)
faster_rcnn/network.py:54: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.fc.weight)


FastRCNN(
  (rpn): RPN(
    (features): DataParallel(
      (module): InceptionFeatureExtraction(
        (Conv2d_1a_3x3): BasicConv2d(
          (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        )
        (Conv2d_2a_3x3): BasicConv2d(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        )
        (Conv2d_2b_3x3): BasicConv2d(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        )
        (Conv2d_3b_1x1): BasicConv2d(
          (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 

In [9]:
# train(None, optimizer, exp_lr_scheduler, net.rpn, 1, 300)
# torch.save(net.rpn.state_dict(), 'rpn.pkl')

In [10]:
param = filter(lambda x: x.requires_grad, net.parameters())
optimizer = SGD(param, lr=1e-3, momentum=0.9, weight_decay=0.0005)
exp_lr_scheduler = StepLR(optimizer, step_size=1000, gamma=0.95)
save_interval = 100

In [11]:
def train(data_gen, optimizer, lr_scheduler ,model, epochs, steps_per_epoch):
    model.train()
    train_loss = 0
    cross_entropy = 0
    loss_box = 0
    rpn_loss = 0
    blobs = data_gen.next()
    max_height = np.max([blob['tensor'].shape[2] for blob in blobs])
    max_width = np.max([blob['tensor'].shape[3] for blob in blobs])
    batch_tensor = torch.Tensor(batch_size, 3, max_height, max_width).fill_(0.)
    total_boxes = 0
    batch_boxes = np.empty((0, 5))
    batch_boxes_index = np.empty((0,), dtype=np.int)
    im_info = np.array([[batch_tensor.shape[2], batch_tensor.shape[3]]])
    for i, blob in enumerate(blobs):
        total_boxes = blob['boxes'].shape[0]
        gt_classes =  blob['gt_classes']
        gt_boxes = np.hstack([blob['boxes'] , gt_classes[:, np.newaxis]])
        batch_boxes = np.vstack((batch_boxes, gt_boxes))
        a =  np.zeros((total_boxes , ), dtype=np.int)
        a.fill(i)
        batch_boxes_index = np.concatenate((batch_boxes_index, a), axis=0)
    
    for step in range(1, steps_per_epoch +1):
        lr_scheduler.step()        
        try:
            cls_prob, bbox_pred, rois = model(batch_tensor, im_info, batch_boxes, batch_boxes_index)
        except Exception as e:
            print batch_boxes_index
            print e
            continue


        cls_data = cls_prob.data.cpu().numpy()
        max_class_idx = np.argmax(cls_data, axis=1)
        loss = model.loss
        cross_entropy += model.cross_entropy.item()
        loss_box += model.loss_box.item()
        train_loss += loss.item()
        rpn_loss += model.rpn.loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % 10 == 0:
            log_text = 'step %d,  loss: %.4f' % (
                step, train_loss /(step))
            print(log_text)

            re_cnt = True


        if step % save_interval == 0:
            torch.save(model.state_dict(), 'faster_model_at_step_%s.pkl' % step)

In [14]:
import sys
for obj in gc.get_objects():
    if torch.is_tensor(obj) and torch.is_tensor(obj.data):
        print(type(obj), obj.size())

(<class 'torch.Tensor'>, torch.Size([]))
(<class 'torch.Tensor'>, torch.Size([]))
(<class 'torch.Tensor'>, torch.Size([900, 4096]))
(<class 'torch.Tensor'>, torch.Size([900, 4096]))
(<class 'torch.Tensor'>, torch.Size([]))
(<class 'torch.Tensor'>, torch.Size([]))
(<class 'torch.Tensor'>, torch.Size([1, 3, 1762, 600]))
(<class 'torch.Tensor'>, torch.Size([1, 3, 1802, 600]))
(<class 'torch.Tensor'>, torch.Size([1, 3, 1828, 600]))
(<class 'torch.Tensor'>, torch.Size([1, 3, 1829, 600]))
(<class 'torch.Tensor'>, torch.Size([1, 3, 1929, 600]))
(<class 'torch.Tensor'>, torch.Size([1, 3, 2000, 600]))
(<class 'torch.Tensor'>, torch.Size([1, 3, 1666, 600]))
(<class 'torch.Tensor'>, torch.Size([1, 3, 1699, 600]))
(<class 'torch.Tensor'>, torch.Size([1, 3, 1754, 600]))
(<class 'torch.Tensor'>, torch.Size([1, 3, 1613, 600]))
(<class 'torch.Tensor'>, torch.Size([1, 3, 1548, 600]))
(<class 'torch.Tensor'>, torch.Size([1, 3, 1517, 600]))
(<class 'torch.Tensor'>, torch.Size([1, 3, 1454, 600]))
(<class 

(<class 'torch.Tensor'>, torch.Size([64]))
(<class 'torch.Tensor'>, torch.Size([64]))
(<class 'torch.Tensor'>, torch.Size([80]))
(<class 'torch.Tensor'>, torch.Size([80]))
(<class 'torch.Tensor'>, torch.Size([32]))
(<class 'torch.Tensor'>, torch.Size([32]))
(<class 'torch.Tensor'>, torch.Size([32]))
(<class 'torch.Tensor'>, torch.Size([32]))
(<class 'torch.Tensor'>, torch.Size([192]))
(<class 'torch.Tensor'>, torch.Size([192]))
(<class 'torch.Tensor'>, torch.Size([128]))
(<class 'torch.Tensor'>, torch.Size([128]))
(<class 'torch.Tensor'>, torch.Size([192]))
(<class 'torch.Tensor'>, torch.Size([192]))
(<class 'torch.Tensor'>, torch.Size([128]))
(<class 'torch.Tensor'>, torch.Size([128]))
(<class 'torch.Tensor'>, torch.Size([128]))
(<class 'torch.Tensor'>, torch.Size([128]))
(<class 'torch.Tensor'>, torch.Size([192]))
(<class 'torch.Tensor'>, torch.Size([192]))
(<class 'torch.Tensor'>, torch.Size([128]))
(<class 'torch.Tensor'>, torch.Size([128]))
(<class 'torch.Tensor'>, torch.Size([192

In [15]:
train(t, optimizer=optimizer,lr_scheduler=exp_lr_scheduler, model=net, epochs=1, steps_per_epoch=500)

step 10,  loss: 4.3338
step 20,  loss: 3.6284
step 30,  loss: 3.1765
step 40,  loss: 2.9306
step 50,  loss: 2.7543
step 60,  loss: 2.5933
step 70,  loss: 2.4785
step 80,  loss: 2.3841
step 90,  loss: 2.3058
step 100,  loss: 2.2431
step 110,  loss: 2.1922
step 120,  loss: 2.1466
step 130,  loss: 2.1162
step 140,  loss: 2.1020
step 150,  loss: 2.0884
step 160,  loss: 2.0782
step 170,  loss: 2.0749
step 180,  loss: 2.0721
step 190,  loss: 2.0700
step 200,  loss: 2.0635
step 210,  loss: 2.0560
step 220,  loss: 2.0483
step 230,  loss: 2.0384
step 240,  loss: 2.0293
step 250,  loss: 2.0196
step 260,  loss: 2.0101
step 270,  loss: 2.0013
step 280,  loss: 1.9938
step 290,  loss: 1.9858
step 300,  loss: 1.9771
step 310,  loss: 1.9699
step 320,  loss: 1.9615
step 330,  loss: 1.9527
step 340,  loss: 1.9446
step 350,  loss: 1.9383
step 360,  loss: 1.9321
step 370,  loss: 1.9249
step 380,  loss: 1.9189
step 390,  loss: 1.9113
step 400,  loss: 1.9060
step 410,  loss: 1.8999
step 420,  loss: 1.8982
s

NameError: name 'blobs' is not defined